In [7]:
import requests
import pandas as pd



url = "https://api.upbit.com/v1/candles/minutes/60"

querystring = {
    "market":"KRW-XRP",
    "count":"200" # 최근부터 몇 개 까지의 데이터를 조회할 것인지
}

response = requests.request("GET", url, params=querystring)

data = response.json()

df = pd.DataFrame(data)
df.head()


,market,candle_date_time_utc,candle_date_time_kst,opening_price,high_price,low_price,trade_price,timestamp,candle_acc_trade_price,candle_acc_trade_volume,unit
0,KRW-XRP,2023-05-30T16:00:00,2023-05-31T01:00:00,675.0,677.0,675.0,676.0,1685462453111,4.891011e+08,7.227596e+05,60
1,KRW-XRP,2023-05-30T15:00:00,2023-05-31T00:00:00,664.0,680.0,662.0,674.0,1685462399312,1.397497e+10,2.083406e+07,60
2,KRW-XRP,2023-05-30T14:00:00,2023-05-30T23:00:00,667.0,668.0,660.0,663.0,1685458800015,6.665962e+09,1.003961e+07,60
3,KRW-XRP,2023-05-30T13:00:00,2023-05-30T22:00:00,667.0,669.0,666.0,666.0,1685455198687,3.258230e+09,4.883128e+06,60
4,KRW-XRP,2023-05-30T12:00:00,2023-05-30T21:00:00,670.0,670.0,665.0,667.0,1685451598548,5.816280e+09,8.717956e+06,60


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

X = df[['opening_price', 'high_price', 'low_price', 'trade_price']]
y = df['candle_acc_trade_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

predictions = model.predict(X_test)

print('RMSE:', mean_squared_error(y_test, predictions, squared=False))


RMSE: 2687100415.220554


In [10]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(12, input_dim=4, kernel_initializer='normal', activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam')

model.fit(X_train, y_train, epochs=1000, batch_size=32)


Epoch 1/1000
5/5 [==============================] - 0s 767us/step - loss: 25908841419378786304.0000
Epoch 2/1000
5/5 [==============================] - 0s 692us/step - loss: 25908841419378786304.0000
Epoch 3/1000
5/5 [==============================] - 0s 704us/step - loss: 25908839220355530752.0000
Epoch 4/1000
5/5 [==============================] - 0s 663us/step - loss: 25908841419378786304.0000
Epoch 5/1000
5/5 [==============================] - 0s 847us/step - loss: 25908839220355530752.0000
Epoch 6/1000
5/5 [==============================] - 0s 816us/step - loss: 25908839220355530752.0000
Epoch 7/1000
5/5 [==============================] - 0s 662us/step - loss: 25908839220355530752.0000
Epoch 8/1000
5/5 [==============================] - 0s 754us/step - loss: 25908839220355530752.0000
Epoch 9/1000
5/5 [==============================] - 0s 632us/step - loss: 25908841419378786304.0000
Epoch 10/1000
5/5 [==============================] - 0s 681us/step - loss: 25908839220355530752.0000

In [11]:
import pyupbit

access_key = "YOUR_ACCESS_KEY"
secret_key = "YOUR_SECRET_KEY"
upbit = pyupbit.Upbit(access_key, secret_key)


In [13]:
# 가장 최근 데이터 가져오기
latest_data = df.iloc[-1]

# 다음 시간의 가격 예측
next_price = model.predict([[latest_data['opening_price'], latest_data['high_price'], latest_data['low_price'], latest_data['trade_price']]])
print(next_price)
# 현재 보유 현금 및 리플 개수 확인
krw_balance = upbit.get_balance(ticker="KRW")
xrp_balance = upbit.get_balance(ticker="KRW-XRP")

# 예측 가격이 현재 가격보다 높고, 현금이 있다면 매수
if next_price > latest_data['trade_price'] and krw_balance > 0:
    upbit.buy_market_order("KRW-XRP", krw_balance)

# 예측 가격이 현재 가격보다 낮고, 리플을 가지고 있다면 매도
elif next_price < latest_data['trade_price'] and xrp_balance > 0:
    upbit.sell_market_order("KRW-XRP", xrp_balance)


[[7571516.]]
TypeError
TypeError


TypeError: '>' not supported between instances of 'NoneType' and 'int'